# Gymnasium 

Gymnasium, es una librería de aprendizaje reforzado, por lo que utiliza el bucle agente-entorno:


<img src="./media/AE_loop.png" width="350px"/>

Para conseguir la interacción correcta entre los elementos que aparecen en la imagen, dentro de gymnasium se encuentran las siguientes clases principales:

## Registry
En gymnasium, se pueden utilizar wrappers es decir, entornos pre-cargados que se pueden incluir en la propia librería o ser externos, para poder realizar pruebas con los mismos. Los distintos entornos pueden ser de terceros o estar incluidos con la librería. En este caso, al ser un tutorial completo cuando se installa gymnasium (*pip install gymnasium[all]*), se instalan los entornos de los siguientes paquetes:
- **Classic Control:** Estos entornos, son estocásticos refiriéndose al estado inicial. Estos entornos, se podrían considerar los más sencillos para resolverlos mediante una política. 
- **Box2d:** Son entornos basados en juegos relacionados con la física, utilizando las físicas de box2d y el renderizado de PyGame.
- **Toy Text:** Entornos muy simples, deiseñados para que tengan un espacio de estados y acciones pequeño, siendo buenos entornos para realizar debug sobre algoritmos de aprendizaje reforzado.
- **Mujoco:** Estos entornos, están preparados para dinámicas multiarticulares con contacto, es decir, orientados al desarrollo en robótica, biomecánica, gráficos, animación... es decir, areas donde una simulación certera y rápida es necesaria. Para ello, es necesario instalar de forma extra el el engine de MuJoCo
- **Atari:** Para poder utilizar estos entornos, es necesario ejecutar el siguiente comando *pip install gymnasium[accept-rom-license]*, el cual instalará AutoROM y descargará las ROMs (únicamente con el comando all o el específico con atari no es necesario). En este caso el espacio de acciones será común para todos los entornos al ser juegos que pertenecen a la misma consola, por lo que siempre se realizan las mismas acciones. Finalmente, estos entornos están simulados por Arcade Learning Enviroment (ALE).

Para mostrar todos los paquetes con los entornos que se encuentran instalados en nuestro sistema, se utilizará la función *pprint_registry()*.

In [3]:
import gymnasium
gymnasium.pprint_registry()

===== classic_control =====
Acrobot-v1                           CartPole-v0                          CartPole-v1
MountainCar-v0                       MountainCarContinuous-v0             Pendulum-v1

===== phys2d =====
CartPoleJax-v0                       CartPoleJax-v1                       PendulumJax-v0

===== box2d =====
BipedalWalker-v3                     BipedalWalkerHardcore-v3             CarRacing-v2
LunarLander-v2                       LunarLanderContinuous-v2

===== toy_text =====
Blackjack-v1                         CliffWalking-v0                      FrozenLake-v1
FrozenLake8x8-v1                     Taxi-v3

===== mujoco =====
Ant-v2                               Ant-v3                               Ant-v4
HalfCheetah-v2                       HalfCheetah-v3                       HalfCheetah-v4
Hopper-v2                            Hopper-v3                            Hopper-v4
Humanoid-v2                          Humanoid-v3                          Humanoid-v4
Humanoid

/home/alberto/anaconda3/envs/gymnasium/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Para poder cargar un entorno wrapped, se utilizará el método *make*, aunque este método tiene muchos parámetros, lo común es indicarle el id del entorno a cargar y el modo de renderizado que se quiere mostrar:
- **gymnasium.make(id-entorno:str, render_mode:str)**
El resto de parámetros podrán ser para indicar un temporizador del número máximo de acciones que se pueden tomar, otro temporizador para indicar cada cuanto se tiene que resetear el entorno, o para compatibilidad entre versiones de gymnasium. Finalmente, este método devuelve una instancia del entorno.

In [35]:
gymnasium.make('CartPole-v1', render_mode="human") 

<TimeLimit<OrderEnforcing<PassiveEnvChecker<CartPoleEnv<CartPole-v1>>>>>

## Enviroment
TODO MIRAR LO DE MULTIAGENTE DE PETTINGZOO (ES OTRA LIBRERÍA): https://github.com/Farama-Foundation/PettingZoo<br> <br>
En esta clase, se encapsula el entorno que puede ser observado por el agente, se va a trabajar con el entorno *LunarLander*, para demostrar todas las propiedades de las clases entorno:

<img src="./media/LunarLander.gif" width="350px"/>

In [4]:
#Carga del entorno para trabajar con la clase
lunarLander = gymnasium.make('LunarLander-v2', render_mode="human") 

Una clase del tipo entorno, consta de los siguientes atributos para facilitar su implementación:

### Atributos

- **action_space:** es el objeto de espacios correspondiente a las acciones válidas.

In [10]:
print(lunarLander.action_space)
print(type(lunarLander.action_space))

Discrete(4)
<class 'gymnasium.spaces.discrete.Discrete'>


Como se puede apreciar *action_space* es un objeto del tipo Discrete, esta clase también es de gymnasium y representa un espacio finito de muchos elementos, concretamente números enteros. <br>
Estos números, posteriormente representarán acciones (a definir por el modelo). Por ejemplo, las acciones del modelo anterior serían:
- **0:** no hacer nada
- **1:** encender el motor izquierdo
- **2:** encender el motor principal (ambos)
- **3:** encender el motor derecho

Para crear un objeto de este tipo, será necesario indicar el número de elementos del espacio (4 en el caso anterior), el elemento más pequeño del espacio (inicio) y opcionalmente, una semilla en caso de querer que sea aleatorio.

In [21]:
d = gymnasium.spaces.Discrete(2,start=0)
print(d)

Discrete(2)


- **observation_space:** es el espacio de observaciones de un entorno.

In [66]:
#print("{}".format(lunarLander.observation_space))
print(lunarLander.observation_space.high)

[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]


El espacio de observaciones, es un objeto tipo *Box*, el cual representa el producto cartesiano de dos intervalos n:
<img src="./media/productoCartesiano.png" width="350px"/>
Los límites de los intervalos pueden ser de una de las siguientes formas, $[a,b], (-\infty,b],[a,\infty) o (-\infty,\infty) $
- 


In [61]:
import numpy as np
b = gymnasium.spaces.Box(low=-2.0, high=np.array([2.0, 4.0]), dtype=np.float32)
print(b)

Box(-2.0, [2. 4.], (2,), float32)



- **reward_range:** es una tupla que hace referencia a la máxima y mínima recompensa posible para el agente en un momento (por defecto es en ($-\infty,+\infty)$


In [7]:
print(lunarLander.reward_range) 

"""No se ha encontrado ningún modelo que tenga parámetros distintos a 
los que se muestran"""


NameError: name 'lunarLander' is not defined

- **spec:** Contiene la información que ha utilizado el método make para inicializar el entorno.


In [83]:
print(lunarLander.spec) 

EnvSpec(id='LunarLander-v2', entry_point='gymnasium.envs.box2d.lunar_lander:LunarLander', reward_threshold=200, nondeterministic=False, max_episode_steps=1000, order_enforce=True, autoreset=False, disable_env_checker=False, apply_api_compatibility=False, kwargs={}, namespace=None, name='LunarLander', version=2)


- **metadata:** Son los metadatos del entorno
- **np_random:** El generador de números aleatorios para el entorno

In [86]:
print(str(lunarLander.metadata) + "\n" ) 
print(lunarLander.np_random)

{'render_modes': ['human', 'rgb_array'], 'render_fps': 50}

Generator(PCG64)


### Métodos
Los métodos principales con los que se puede trabajar con el entorno son:

# TODO preguntar que pasa con el reset al profesor
- **reset(self, seed) &rarr; tuple[ObsType, dict[str, Any]]:** El entorno vuelve al estado inicial y devuelve la primera observación del agente e información como métricas, debug... este médoto, también renderiza el entorno al final de su ejecución

En el caso de este entorno, es un vector de 8 dimensiones y cada una representa:
1. Coordenada x de la nave
2. Coordenada y de la nave
3. Velocidad linear en x de la nave
4. Velocidad linear en y de la nave
5. Ángulo de la nave
6. Velocidad angular de la nave
7. Motor izquierdo en contacto con el suelo (booleano)
8. Motor derecho en contacto con el suelo (booleano)


In [6]:
observation, info = lunarLander.reset()

print("La coordenada x de la nave es: {}".format(observation[0]))
print("La coordenada y de la nave: {}".format(observation[1]))
print("La velocidad linear en x de la nave es: {}".format(observation[2]))
print("La velocidad linear en y de la nave es: {}".format(observation[3]))
print("El ángulo de la nave es: {}".format(observation[4]))
print("La velocidad angular de la nave es: {}".format(observation[5]))
print("Motor izquierdo en contacto?: {}".format(observation[6]))
print("Motor derecho en contacto?: {}".format(observation[7]))
print("Informacion extra: {}".format(info))


La coordenada x de la nave es: -0.006235313601791859
La coordenada y de la nave: 1.4109978675842285
La velocidad linear en x de la nave es: -0.6315816044807434
La velocidad linear en y de la nave es: 0.0034475468564778566
El ángulo de la nave es: 0.007231917232275009
La velocidad angular de la nave es: 0.14306268095970154
Motor izquierdo en contacto?: 0.0
Motor derecho en contacto?: 0.0
Informacion extra: {}


- **step(self, action: ActType) &rarr; tuple[ObsType, SupportsFloat, bool, bool, dict[str, Any]]:** Actualiza el entorno con la acción pasada por parámetros, devolviendo la siguiente observación del agente, la recompensa obtenida, si la ejecución del entorno ha terminado debido a la última acción o se ha truncado (timeout o el agente se encuentra fuera de los límites) e información del entorno sobre la acción e información de depuración.

In [8]:
observation, reward, terminated, truncated, info = lunarLander.step(0)
#No se hace nada 
print("No se hace nada (en este caso unicamente se cae la nave): \n\n")
print("La coordenada x de la nave es: {}".format(observation[0]))
print("La coordenada y de la nave: {}".format(observation[1]))
print("La velocidad linear en x de la nave es: {}".format(observation[2]))
print("La velocidad linear en y de la nave es: {}".format(observation[3]))
print("El ángulo de la nave es: {}".format(observation[4]))
print("La velocidad angular de la nave es: {}".format(observation[5]))
print("Motor izquierdo en contacto?: {}".format(observation[6]))
print("Recompensa obtenida: {}".format(reward))
print("Terminado?: {}".format(terminated))
print("Truncado?: {}".format(truncated))
print("Info extra: {}".format(info))



observation, reward, terminated, truncated, info = lunarLander.step(2) 
#Se encienden los dos motores
print("\n\nSe encienden los dos motores: \n\n")
print("La coordenada x de la nave es: {}".format(observation[0]))
print("La coordenada y de la nave: {}".format(observation[1]))
print("La velocidad linear en x de la nave es: {}".format(observation[2]))
print("La velocidad linear en y de la nave es: {}".format(observation[3]))
print("El ángulo de la nave es: {}".format(observation[4]))
print("La velocidad angular de la nave es: {}".format(observation[5]))
print("Motor izquierdo en contacto?: {}".format(observation[6]))
print("Motor derecho en contacto?: {}".format(observation[7]))
print("Recompensa obtenida: {}".format(reward))
print("Terminado?: {}".format(terminated))
print("Truncado?: {}".format(truncated))
print("Info extra: {}".format(info))


No se hace nada (en este caso unicamente se cae la nave): 


La coordenada x de la nave es: -0.025155162438750267
La coordenada y de la nave: 1.4104490280151367
La velocidad linear en x de la nave es: -0.6409107446670532
La velocidad linear en y de la nave es: -0.014545747078955173
El ángulo de la nave es: 0.027537941932678223
La velocidad angular de la nave es: 0.13237515091896057
Motor izquierdo en contacto?: 0.0
Recompensa obtenida: -0.646090397765505
Terminado?: False
Truncado?: False
Info extra: {}


Se encienden los dos motores: 


La coordenada x de la nave es: -0.03164844587445259
La coordenada y de la nave: 1.4103515148162842
La velocidad linear en x de la nave es: -0.6553610563278198
La velocidad linear en y de la nave es: -0.00445215729996562
El ángulo de la nave es: 0.03350500762462616
La velocidad angular de la nave es: 0.11935259401798248
Motor izquierdo en contacto?: 0.0
Motor derecho en contacto?: 0.0
Recompensa obtenida: -2.330074567044238
Terminado?: False
Truncado?: 

En este caso, se puede apreciar que la coordenada *y* desciende la primera vez 0.5 al no realizar ninguna acción. <br> <br>
Después de realizar el segundo paso (encender ambos motores), el descenso en el eje *y* es de 0.1 (mucho menor que en la primera ejecución), por lo que se demuestra que se ha ejecutado la acción, ya que al encenderse ambos motores no se ha producido un descenso tan grande como en la primera.

- **render(self) &rarr; RenderFrame | list[RenderFrame] | None:** Renderiza el entorno para que se pueda visualizar. Es necesario que se especifique el tipo de renderizado a la hora de realizar el *make* 

In [37]:
lunarLander.reset()
lunarLander.render()

- **close(self):** cierra el entorno y acaba la ejecución (similar a cuandos se cierra un fichero).

In [10]:
lunarLander.close()

Una vez vistos todos los métodos y atributos, se va a probar a realizar una pequeña ejecución sobre el agente y el entorno en la que se van a renderizar unos pasos para comprobar gráficamente como interaccionan los elementos:

In [9]:
import time
lunarLander.reset()
lunarLander.step(0) #no se hace nada (desciende en y)
lunarLander.render()
time.sleep(5)

lunarLander.step(1) #se enciende el motor izquierdo (giro angular hacia la derecha)
lunarLander.render()
time.sleep(5)
lunarLander.close()

## Wrappers
Esta clase, sirve para poder modificar un entorno existente sin tener que modificar el código directamente, haciendo que sean modulares, es decir, son una especie de intermediario entre el código fuente del entorno y el programador. Al realizar el método *make*, los métodos se generan por defecto. Para poder modificar un entorno, será necesario inicializar el entorno base para luego modificarlo con los parámetros extra. Un ejemplo en el que se modifican las acciones sería:

In [6]:
import numpy as np
import gymnasium as gym
from gymnasium.wrappers import RescaleAction
env = gym.make("BipedalWalker-v3")
print(env.action_space)

min_action = np.array([0.0, 0.0, 0.0, 0.0])
max_action = np.array([0.45, 0.5, 1.0, 0.75])
env = RescaleAction(env, min_action=min_action, max_action=max_action)
print(env.action_space)


Box(-1.0, 1.0, (4,), float32)
Box(-1.0, 1.0, (4,), float32)


## Vectors

Esta clase, sirve para poder ejecutar copias del mismo entorno al mismo tiempo. Con esta clase, se podría aumentar la velocidad del entrenamiento en algunos casos al poder ejecutar paralelamente muchos casos. A estos entornos, se les denominarán *entornos vectoriales*. Finalmente, tienen los siguientes atributos:
- **num_envs:** El número de sub-entornos que se encuentran en el vector.
- **observation_space:** espacio de observaciones del entorno vectorial.
- **single_observation_space:** espacio de observaciones para un único entorno del vector.
- **action_space:** el espacio de acciones agrupado del entorno vectorial.
- **single_action_space:** el espacio de acciones de un único entorno del vector.

### Métodos
- **reset():** Igual que el anterior, pero resetea todos los entornos paralelos, devolviendo una agrupación de observaciones e información del entorno vectorial.
- **step():** Igual que el anterior, per se le pasa un conjunto de acciones con el que cada entorno del vector tomará una acción.
- **close():** Cierra todos los entornos del vector

<br>
Para crear un entorno vectorial, se hará de la misma forma que en el anterior pero con un pequeño cambio:

In [8]:
import gymnasium as gym
env = gym.vector.make('CartPole-v1', num_envs=3)
env.reset()

(array([[-0.01876395,  0.0314865 ,  0.0395258 , -0.01786517],
        [-0.02476006,  0.0216246 , -0.01668862,  0.04014067],
        [ 0.01843816, -0.04881953, -0.04991993, -0.04886381]],
       dtype=float32),
 {})

Finalmente, habrá dos tipos de entornos vectoriales:
- Asíncronos (Async Vector Env). Múltiples entornos en paralelo
- Síncronos (Sync Vector Env). Múltiples entornos seguidos